<div style="background-color: #04D7FD; padding: 20px; text-align: left;">
    <h1 style="color: #000000; font-size: 36px; margin: 0;">Data Processing for RAG with Data Prep Kit (Python)</h1>
    
</div>


## Before Running the notebook

Please complete [setting up python dev environment](./setup-python-dev-env.md)

## Overview

This notebook will process PDF documents as part of RAG pipeline

TODO : update diagram

![](media/rag-overview-2.png)

This notebook will perform steps 1, 2 and 3 in RAG pipeline.

Here are the processing steps:

- **pdf2parquet** : Extract text (in markdown format) from PDF and store them as parquet files
- **Exact Dedup**: Documents with exact content are filtered out
- **Chunk documents**: Split the PDFs into 'meaningful sections' (paragraphs, sentences ..etc)
- **Text encoder**: Convert chunks into vectors using embedding models

## Step-1: Configuration

In [1]:
from my_config import MY_CONFIG

## Step-2:  Data

We will use white papers  about LLMs.  

- [Granite Code Models](https://arxiv.org/abs/2405.04324)
- [Attention is all you need](https://arxiv.org/abs/1706.03762)

You can of course substite your own data below

### 2.1 - Download data

In [2]:
import os, sys
import shutil
from utils import download_file

shutil.rmtree(MY_CONFIG.INPUT_DATA_DIR, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.INPUT_DATA_DIR, exist_ok=True)
print ("✅ Cleared input directory")
 
download_file (url = 'https://arxiv.org/pdf/1706.03762', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'attention.pdf' ))
download_file (url = 'https://arxiv.org/pdf/2405.04324', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'granite.pdf' ))
# download_file (url = 'https://arxiv.org/pdf/2405.04324', local_file = os.path.join(MY_CONFIG.INPUT_DATA_DIR, 'granite2.pdf' )) # duplicate


✅ Cleared input directory

input/attention.pdf (2.22 MB) downloaded successfully.

input/granite.pdf (1.27 MB) downloaded successfully.


### 2.2 - Set input/output path variables for the pipeline

In [3]:
import os, sys
import shutil

if not os.path.exists(MY_CONFIG.INPUT_DATA_DIR ):
    raise Exception (f"❌ Input folder MY_CONFIG.INPUT_DATA_DIR = '{MY_CONFIG.INPUT_DATA_DIR}' not found")

output_parquet_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '01_parquet_out')
output_exact_dedupe_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '02_dedupe_out')
output_chunk_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '03_chunk_out')
output_embeddings_dir = os.path.join (MY_CONFIG.OUTPUT_FOLDER, '04_embeddings_out')

## clear output folder
shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER, ignore_errors=True)
shutil.os.makedirs(MY_CONFIG.OUTPUT_FOLDER, exist_ok=True)

print ("✅ Cleared output directory")

✅ Cleared output directory


## Step-3: pdf2parquet -  Convert data from PDF to Parquet

This step is reading the input folder containing all PDF files and ingest them in a parquet table using the [Docling package](https://github.com/DS4SD/docling).
The documents are converted into a JSON format which allows to easily chunk it in the later steps.



### 3.1 - Execute 

In [4]:
%%time 

from dpk_pdf2parquet.transform_python import Pdf2Parquet

STAGE = 1 
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{MY_CONFIG.INPUT_DATA_DIR}' --> output='{output_parquet_dir}'\n", flush=True)

result = Pdf2Parquet(input_folder= MY_CONFIG.INPUT_DATA_DIR,
               output_folder= output_parquet_dir,
               data_files_to_use=['.pdf'],
               pdf2parquet_contents_type='application/json'
            #    pdf2parquet_contents_type='text/markdown'
               ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-1: Processing input='input' --> output='output/01_parquet_out'



23:46:12 INFO - pdf2parquet parameters are : {'batch_size': -1, 'artifacts_path': None, 'contents_type': <pdf2parquet_contents_types.JSON: 'application/json'>, 'do_table_structure': True, 'do_ocr': True, 'ocr_engine': <pdf2parquet_ocr_engine.EASYOCR: 'easyocr'>, 'bitmap_area_threshold': 0.05, 'pdf_backend': <pdf2parquet_pdf_backend.DLPARSE_V2: 'dlparse_v2'>, 'double_precision': 8}
23:46:12 INFO - pipeline id pipeline_id
23:46:12 INFO - code location None
23:46:12 INFO - data factory data_ is using local data access: input_folder - input output_folder - output/01_parquet_out
23:46:12 INFO - data factory data_ max_files -1, n_sample -1
23:46:12 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.pdf'], files to checkpoint ['.parquet']
23:46:12 INFO - orchestrator pdf2parquet started at 2025-01-09 23:46:12
23:46:12 INFO - Number of files is 2, source profile {'max_file_size': 2.112621307373047, 'min_file_size': 1.21464157104

Fetching 9 files:   0%|          | 0/9 [00:00<?, ?it/s]

ERR#: COULD NOT CONVERT TO RS THIS TABLE TO COMPUTE SPANS


23:46:31 INFO - Completed 1 files (50.0%) in 0.247 min
23:47:00 INFO - Completed 2 files (100.0%) in 0.741 min
23:47:00 INFO - Done processing 2 files, waiting for flush() completion.
23:47:00 INFO - done flushing in 0.0 sec
23:47:00 INFO - Completed execution in 0.812 min, execution result 0


✅ Stage:1 completed successfully
CPU times: user 2min 4s, sys: 8.53 s, total: 2min 13s
Wall time: 53 s


### 3.2 -  Inspect Generated output

Here we should see one entry per input file processed

In [5]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_parquet_dir)

# print ("Output dimensions (rows x columns)= ", output_df.shape)

output_df.head(5)

## To display certain columns
#parquet_df[['column1', 'column2', 'column3']].head(5)

,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename
0,attention.pdf,"{""schema_name"":""DoclingDocument"",""version"":""1....",15,6,147,6c96cda3-508a-4e78-9a75-e5c3d658eeae,2949302674760005271,pdf,d92ccac154a730b012ad65243218741b6f59dce60ae9dd...,341129,2025-01-09T23:46:31.099708,14.772966,attention.pdf
1,granite.pdf,"{""schema_name"":""DoclingDocument"",""version"":""1....",28,19,295,26ccd99d-04a3-441f-b4a4-2c8717932dcd,3127757990743433032,pdf,ce45cd417581ff3e94d57b1a7253e301dbebd8f2deeabc...,1860416,2025-01-09T23:47:00.726998,29.563914,granite.pdf


## Step-4: Eliminate Duplicate Documents

We have 2 duplicate documnets here : `granite.pdf` and `granite2.pdf`.

Note how the `hash` for these documents are same.

We are going to perform **de-dupe**

On the content of each document, a SHA256 hash is computed, followed by de-duplication of record having identical hashes.

[Dedupe transform documentation](https://github.com/IBM/data-prep-kit/blob/dev/transforms/universal/ededup/README.md)

### 4.1 - Execute 

In [6]:
%%time 

from dpk_ededup.transform_python import Ededup

STAGE = 2
input_df = read_parquet_files_as_df(output_parquet_dir)  ## for debug purposes
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_parquet_dir}' --> output='{output_exact_dedupe_dir}'\n", flush=True)

result = Ededup(input_folder=output_parquet_dir,
    output_folder=output_exact_dedupe_dir,
    ededup_doc_column="contents",
    ededup_doc_id_column="hash"
    ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-2: Processing input='output/01_parquet_out' --> output='output/02_dedupe_out'



23:47:00 INFO - exact dedup params are {'doc_column': 'contents', 'doc_id_column': 'hash', 'use_snapshot': False, 'snapshot_directory': None}
23:47:00 INFO - pipeline id pipeline_id
23:47:00 INFO - code location None
23:47:00 INFO - data factory data_ is using local data access: input_folder - output/01_parquet_out output_folder - output/02_dedupe_out
23:47:00 INFO - data factory data_ max_files -1, n_sample -1
23:47:00 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
23:47:00 INFO - orchestrator ededup started at 2025-01-09 23:47:00
23:47:00 INFO - Number of files is 2, source profile {'max_file_size': 0.13184070587158203, 'min_file_size': 0.03802204132080078, 'total_file_size': 0.1698627471923828}
23:47:00 INFO - Starting from the beginning
23:47:00 INFO - Completed 1 files (50.0%) in 0.0 min
23:47:00 INFO - Completed 2 files (100.0%) in 0.0 min
23:47:00 INFO - Done process

✅ Stage:2 completed successfully
CPU times: user 41.1 ms, sys: 9.31 ms, total: 50.4 ms
Wall time: 50.6 ms


### 4.2 - Inspect Generated output

We would see 2 documents: `attention.pdf`  and `granite.pdf`.  The duplicate `granite.pdf` has been filtered out!

In [7]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_exact_dedupe_dir)

# print ("Input data dimensions (rows x columns)= ", input_df.shape)
# print ("Output data dimensions (rows x columns)= ", output_df.shape)
print (f"Input files before exact dedupe : {input_df.shape[0]:,}")
print (f"Output files after exact dedupe : {output_df.shape[0]:,}")
print ("Duplicate files removed :  ", (input_df.shape[0] - output_df.shape[0]))

output_df.sample(min(3, output_df.shape[0]))

Input files before exact dedupe : 2
Output files after exact dedupe : 2
Duplicate files removed :   0


,filename,contents,num_pages,num_tables,num_doc_elements,document_id,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,removed
1,granite.pdf,"{""schema_name"":""DoclingDocument"",""version"":""1....",28,19,295,26ccd99d-04a3-441f-b4a4-2c8717932dcd,3127757990743433032,pdf,ce45cd417581ff3e94d57b1a7253e301dbebd8f2deeabc...,1860416,2025-01-09T23:47:00.726998,29.563914,granite.pdf,[]
0,attention.pdf,"{""schema_name"":""DoclingDocument"",""version"":""1....",15,6,147,6c96cda3-508a-4e78-9a75-e5c3d658eeae,2949302674760005271,pdf,d92ccac154a730b012ad65243218741b6f59dce60ae9dd...,341129,2025-01-09T23:46:31.099708,14.772966,attention.pdf,[]


##  Step-5: Doc chunks

Split the documents in chunks.

[Chunking transform documentation](https://github.com/IBM/data-prep-kit/blob/dev/transforms/language/doc_chunk/README.md)

### 5.1 - Execute 

In [8]:
%%time

from dpk_doc_chunk.transform_python import DocChunk

STAGE = 3
input_df = read_parquet_files_as_df(output_exact_dedupe_dir)  ## for debug purposes
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_exact_dedupe_dir}' --> output='{output_chunk_dir}'\n", flush=True)


result = DocChunk(input_folder=output_exact_dedupe_dir,
        output_folder=output_chunk_dir,
        # doc_chunk_chunking_type= "li_markdown",
        doc_chunk_chunking_type= "dl_json",
        doc_chunk_chunk_size_tokens = 256,
        doc_chunk_chunk_overlap_tokens=20
        ).transform()

if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-3: Processing input='output/02_dedupe_out' --> output='output/03_chunk_out'



23:47:02 INFO - doc_chunk parameters are : {'chunking_type': 'dl_json', 'content_column_name': 'contents', 'doc_id_column_name': 'document_id', 'output_chunk_column_name': 'contents', 'output_source_doc_id_column_name': 'source_document_id', 'output_jsonpath_column_name': 'doc_jsonpath', 'output_pageno_column_name': 'page_number', 'output_bbox_column_name': 'bbox', 'chunk_size_tokens': 128, 'chunk_overlap_tokens': 30, 'dl_min_chunk_len': None}
23:47:02 INFO - pipeline id pipeline_id
23:47:02 INFO - code location None
23:47:02 INFO - data factory data_ is using local data access: input_folder - output/02_dedupe_out output_folder - output/03_chunk_out
23:47:02 INFO - data factory data_ max_files -1, n_sample -1
23:47:02 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
23:47:02 INFO - orchestrator doc_chunk started at 2025-01-09 23:47:02
23:47:02 INFO - Number of files is 2, sou

✅ Stage:3 completed successfully
CPU times: user 1.22 s, sys: 183 ms, total: 1.4 s
Wall time: 1.74 s


### 5.2 - Inspect Generated output

We would see documents are split into many chunks

In [9]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_chunk_dir)

print (f"Files processed : {input_df.shape[0]:,}")
print (f"Chunks created : {output_df.shape[0]:,}")

# print ("Input data dimensions (rows x columns)= ", input_df.shape)
# print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Files processed : 2
Chunks created : 390


,filename,num_pages,num_tables,num_doc_elements,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,removed,source_document_id,contents,doc_jsonpath,page_number,bbox,document_id
153,granite.pdf,28,19,295,3127757990743433032,pdf,ce45cd417581ff3e94d57b1a7253e301dbebd8f2deeabc...,1860416,2025-01-09T23:47:00.726998,29.563914,granite.pdf,[],26ccd99d-04a3-441f-b4a4-2c8717932dcd,The pretraining code data was sourced from a c...,#/texts/40,4,"[107.2206955, 373.6947937, 505.50299072, 550.2...",299b90813718972af9e698c8d96b547ea70b5ceab70b41...
306,granite.pdf,28,19,295,3127757990743433032,pdf,ce45cd417581ff3e94d57b1a7253e301dbebd8f2deeabc...,1860416,2025-01-09T23:47:00.726998,29.563914,granite.pdf,[],26ccd99d-04a3-441f-b4a4-2c8717932dcd,· MATH+Py solving MATH task by writing a Pytho...,#/texts/205,19,"[132.84700012, 218.51342773, 504.05380249, 251...",49af5631f23b3c372f170bf9155c9f50da690a247e803f...
301,granite.pdf,28,19,295,3127757990743433032,pdf,ce45cd417581ff3e94d57b1a7253e301dbebd8f2deeabc...,1860416,2025-01-09T23:47:00.726998,29.563914,granite.pdf,[],26ccd99d-04a3-441f-b4a4-2c8717932dcd,Table 15: Performance on 4 chain-of-thought ma...,#/texts/201,19,"[118.96057129, 699.65802002, 492.17700195, 710...",d5ce3a0057d0e6e0013faeb51661999feb845688f002bf...


## Step-6:   Calculate Embeddings for Chunks

we will calculate embeddings for each chunk using an open source embedding model

[Embeddings / Text Encoder documentation](https://github.com/IBM/data-prep-kit/blob/dev/transforms/language/text_encoder/README.md)

### 6.1 - Execute

In [10]:
%%time 

from dpk_text_encoder.transform_python import TextEncoder

STAGE  = 4
input_df = read_parquet_files_as_df(output_chunk_dir)  ## for debug purposes
print (f"🏃🏼 STAGE-{STAGE}: Processing input='{output_chunk_dir}' --> output='{output_embeddings_dir}'\n", flush=True)


result = TextEncoder(input_folder= output_chunk_dir, 
               output_folder= output_embeddings_dir, 
               text_encoder_model_name = MY_CONFIG.EMBEDDING_MODEL
               ).transform()
if result == 0:
    print (f"✅ Stage:{STAGE} completed successfully")
else:
    raise Exception (f"❌ Stage:{STAGE}  failed")

🏃🏼 STAGE-4: Processing input='output/03_chunk_out' --> output='output/04_embeddings_out'



23:47:03 INFO - text_encoder parameters are : {'content_column_name': 'contents', 'output_embeddings_column_name': 'embeddings', 'model_name': 'sentence-transformers/all-MiniLM-L6-v2'}
23:47:03 INFO - pipeline id pipeline_id
23:47:03 INFO - code location None
23:47:03 INFO - data factory data_ is using local data access: input_folder - output/03_chunk_out output_folder - output/04_embeddings_out
23:47:03 INFO - data factory data_ max_files -1, n_sample -1
23:47:03 INFO - data factory data_ Not using data sets, checkpointing False, max files -1, random samples -1, files to use ['.parquet'], files to checkpoint ['.parquet']
23:47:03 INFO - orchestrator text_encoder started at 2025-01-09 23:47:03
23:47:03 INFO - Number of files is 2, source profile {'max_file_size': 0.06411361694335938, 'min_file_size': 0.03104114532470703, 'total_file_size': 0.0951547622680664}
23:47:06 INFO - Completed 1 files (50.0%) in 0.009 min
23:47:07 INFO - Completed 2 files (100.0%) in 0.03 min
23:47:07 INFO - Do

✅ Stage:4 completed successfully
CPU times: user 2.51 s, sys: 315 ms, total: 2.83 s
Wall time: 5.04 s


### 6.2 - Inspect Generated output

In [11]:
from utils import read_parquet_files_as_df

output_df = read_parquet_files_as_df(output_embeddings_dir)

print ("Input data dimensions (rows x columns)= ", input_df.shape)
print ("Output data dimensions (rows x columns)= ", output_df.shape)

output_df.sample(min(3, output_df.shape[0]))

Input data dimensions (rows x columns)=  (390, 18)
Output data dimensions (rows x columns)=  (390, 19)


,filename,num_pages,num_tables,num_doc_elements,document_hash,ext,hash,size,date_acquired,pdf_convert_time,source_filename,removed,source_document_id,contents,doc_jsonpath,page_number,bbox,document_id,embeddings
188,granite.pdf,28,19,295,3127757990743433032,pdf,ce45cd417581ff3e94d57b1a7253e301dbebd8f2deeabc...,1860416,2025-01-09T23:47:00.726998,29.563914,granite.pdf,[],26ccd99d-04a3-441f-b4a4-2c8717932dcd,"We use AdamW optimizer (Kingma & Ba, 2017) wit...",#/texts/83,7,"[107.10358429, 569.9118042, 504.39199829, 651....",6e5757149716cad61caac3bcb92b0e53df860f4b313b8f...,"[-0.06271278, -0.024036769, 0.019579308, 0.036..."
286,granite.pdf,28,19,295,3127757990743433032,pdf,ce45cd417581ff3e94d57b1a7253e301dbebd8f2deeabc...,1860416,2025-01-09T23:47:00.726998,29.563914,granite.pdf,[],26ccd99d-04a3-441f-b4a4-2c8717932dcd,"From Table 12, we make the following observati...",#/texts/182,17,"[107.07395935, 356.62701416, 505.74499512, 411...",0a7ccb8e35838ffedcb5ce7e0737fe7b0866d65f02dcc5...,"[-0.056604, -0.02220467, -0.04024029, -0.03403..."
86,attention.pdf,15,6,147,2949302674760005271,pdf,d92ccac154a730b012ad65243218741b6f59dce60ae9dd...,341129,2025-01-09T23:46:31.099708,14.772966,attention.pdf,[],6c96cda3-508a-4e78-9a75-e5c3d658eeae,To evaluate the importance of different compon...,#/texts/108,8,"[107.07932281, 91.68579102, 504.0, 113.67138672]",f878a8524f8524bbfd5dd01bfdddeda9e6de18abdfd848...,"[-0.020042712, -0.00294097, -0.003946473, -0.0..."


## Step-7: Copy output to final output dir

In [12]:
import shutil

shutil.rmtree(MY_CONFIG.OUTPUT_FOLDER_FINAL, ignore_errors=True)
shutil.copytree(src=output_embeddings_dir, dst=MY_CONFIG.OUTPUT_FOLDER_FINAL)

print (f"✅ Copied output from '{output_embeddings_dir}' --> '{MY_CONFIG.OUTPUT_FOLDER_FINAL}'")

✅ Copied output from 'output/04_embeddings_out' --> 'output/output_final'
